### 데이터셋

In [1]:
import pandas as pd
import numpy as np

# 센서 로그 데이터
sensor_logs = pd.DataFrame({
    'log_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'machine_id': ['M001', 'M001', 'M002', 'M002', 'M001', 'M003', 'M003', 'M002'],
    'timestamp': ['2024-01-01 09:00', '2024-01-01 09:05', '2024-01-01 09:00', 
                  '2024-01-01 09:10', '2024-01-01 09:10', '2024-01-01 09:00',
                  '2024-01-01 09:05', '2024-01-01 09:15'],
    'temperature': [35.2, 38.5, 42.1, np.nan, 36.0, 55.0, 58.3, 40.0],
    'status': ['normal', 'normal', 'warning', 'warning', 'normal', 'critical', 'critical', 'normal']
})

# 기계 정보
machines = pd.DataFrame({
    'machine_id': ['M001', 'M002', 'M003'],
    'machine_name': ['프레스A', '프레스B', '용접기'],
    'max_temp': [40.0, 45.0, 60.0]  # 허용 최대 온도
})

print("=== sensor_logs ===")
print(sensor_logs)
print("\n=== machines ===")
print(machines)

=== sensor_logs ===
   log_id machine_id         timestamp  temperature    status
0       1       M001  2024-01-01 09:00         35.2    normal
1       2       M001  2024-01-01 09:05         38.5    normal
2       3       M002  2024-01-01 09:00         42.1   warning
3       4       M002  2024-01-01 09:10          NaN   warning
4       5       M001  2024-01-01 09:10         36.0    normal
5       6       M003  2024-01-01 09:00         55.0  critical
6       7       M003  2024-01-01 09:05         58.3  critical
7       8       M002  2024-01-01 09:15         40.0    normal

=== machines ===
  machine_id machine_name  max_temp
0       M001         프레스A      40.0
1       M002         프레스B      45.0
2       M003          용접기      60.0


### temperature 결측치를 해당 기계의 평균 온도로 채우기

In [12]:
import pandas as pd
import numpy as np

# 1. 샘플 데이터
df = pd.DataFrame({
    'machine_id': ['M001', 'M001', 'M002', 'M002'],
    'temperature': [30, 40, np.nan, 50] # M001평균: 35, M002평균: 50(NaN제외하면 50하나니까)
})

print("--- 1. 원본 ---")
print(df)

# 2. 참조표 만들기 (Dictionary)
# M001 -> 35.0, M002 -> 50.0
mean_dict = df.groupby('machine_id')['temperature'].mean()
print("\n--- 2. 참조표 (Lookup Table) ---")
print(mean_dict)

# 3. 매핑하기 (중간 단계 확인! 중요!)
# 원본 데이터 옆에 '평균값'을 쫙 깔아버림
mapped_values = df['machine_id'].map(mean_dict)
print("\n--- 3. 매핑 결과 (가상의 열) ---")
print(mapped_values)
# 결과: [35.0, 35.0, 50.0, 50.0]

# 4. 채우기
# 원본이 NaN인 곳만, mapped_values에서 가져옴
df['temperature'] = df['temperature'].fillna(mapped_values)
print("\n--- 4. 최종 결과 ---")
print(df)

--- 1. 원본 ---
  machine_id  temperature
0       M001         30.0
1       M001         40.0
2       M002          NaN
3       M002         50.0

--- 2. 참조표 (Lookup Table) ---
machine_id
M001    35.0
M002    50.0
Name: temperature, dtype: float64

--- 3. 매핑 결과 (가상의 열) ---
0    35.0
1    35.0
2    50.0
3    50.0
Name: machine_id, dtype: float64

--- 4. 최종 결과 ---
  machine_id  temperature
0       M001         30.0
1       M001         40.0
2       M002         50.0
3       M002         50.0


In [ ]:
sensor_logs['temperature_mean'] = sensor_logs.groupby('machine_id',as_index=False)['temperature'].transform('mean')
print(sensor_logs['temperature'])
print(sensor_logs['temperature'].fillna(sensor_logs['temperature_mean'])) 

# 🐚 이게 옆에꺼가 채워지는건 뭐지 ? 어찌 되는거지 ?  -> 평균 구하는건 했는데 nan에 들어가려면 모델이 동일해야하고 이런게 나는 조건이 붙는다고 생각했는데
# 🐚 fillna()가 수행할떄 같은번호(index())를 기준으로 붙인다  fillna()는 "같은 인덱스끼리 매칭"해서 채우고, map()은 "값을 사전처럼 변환"한다. NaN이든 아니든 상관없이, machine_id만 보고 변환!

# 🐚 다른방법들은 
# # 1단계: 참조표(Reference Table) 만들기
# 결과: {'M001': 36.5, 'M002': 41.8, ...} 형태의 Series가 생성됨 --> mean_dict = sensor_logs.groupby('machine_id')['temperature'].mean()
# 2단계: map으로 매핑해서 채우기
# 해석: "내 machine_id를 보고, 위에서 만든 표(mean_dict)에서 값을 찾아와서(fillna) 채워라" 
# sensor_logs['temperature'] = sensor_logs['temperature'].fillna(sensor_logs['machine_id'].map(mean_dict))) 
# --> 매핑. 매핑 개념 정확히 모르겠음 항상. 


SyntaxError: unmatched ')' (2065460599.py, line 18)

### status가 'critical'인 로그만 필터링

In [3]:
sensor_logs_filter = sensor_logs[sensor_logs['status'] == 'critical'] # 🐚 순간적으로 이거 행으로 나오는건지 아니면 출력이 어떤식으로 되는지 머리속에 안그려짐
sensor_logs_filter

,log_id,machine_id,timestamp,temperature,status,temperature_mean
5,6,M003,2024-01-01 09:00,55.0,critical,56.65
6,7,M003,2024-01-01 09:05,58.3,critical,56.65


### 기계(machine_id)별 평균 온도 구하기 (새 테이블)

In [4]:
machine_id_temp_mean = sensor_logs.groupby('machine_id',as_index=False)['temperature'].mean()
machine_id_temp_mean

,machine_id,temperature
0,M001,36.566667
1,M002,41.050000
2,M003,56.650000


### sensor_logs에 기계 이름(machine_name)과 허용 최대 온도(max_temp) 붙이기

In [5]:
# sensor를 기준으로 하여금 pd.merge(sensor_logs,machines[['machine_name','max_temp']],how='left',on='machine_id')

sensor_logs_merge = pd.merge(sensor_logs,machines[['machine_id','machine_name','max_temp']],how='left',on='machine_id') # 🐚 조건은 무조건 합치는컬럼내에 붙어야함.
sensor_logs_merge.sort_values('machine_id',ascending=True) # 조회만한거. 반영x 

,log_id,machine_id,timestamp,temperature,status,temperature_mean,machine_name,max_temp
0,1,M001,2024-01-01 09:00,35.2,normal,36.566667,프레스A,40.0
1,2,M001,2024-01-01 09:05,38.5,normal,36.566667,프레스A,40.0
4,5,M001,2024-01-01 09:10,36.0,normal,36.566667,프레스A,40.0
2,3,M002,2024-01-01 09:00,42.1,warning,41.050000,프레스B,45.0
3,4,M002,2024-01-01 09:10,NaN,warning,41.050000,프레스B,45.0
7,8,M002,2024-01-01 09:15,40.0,normal,41.050000,프레스B,45.0
5,6,M003,2024-01-01 09:00,55.0,critical,56.650000,용접기,60.0
6,7,M003,2024-01-01 09:05,58.3,critical,56.650000,용접기,60.0


### 4번 결과에서 "온도 초과 여부" 컬럼 추가 (temperature > max_temp)

In [6]:
sensor_logs_merge['over_temp'] = np.where(sensor_logs_merge['temperature'] > sensor_logs_merge['max_temp'],'비상','안전')
sensor_logs_merge

,log_id,machine_id,timestamp,temperature,status,temperature_mean,machine_name,max_temp,over_temp
0,1,M001,2024-01-01 09:00,35.2,normal,36.566667,프레스A,40.0,안전
1,2,M001,2024-01-01 09:05,38.5,normal,36.566667,프레스A,40.0,안전
2,3,M002,2024-01-01 09:00,42.1,warning,41.050000,프레스B,45.0,안전
3,4,M002,2024-01-01 09:10,NaN,warning,41.050000,프레스B,45.0,안전
4,5,M001,2024-01-01 09:10,36.0,normal,36.566667,프레스A,40.0,안전
5,6,M003,2024-01-01 09:00,55.0,critical,56.650000,용접기,60.0,안전
6,7,M003,2024-01-01 09:05,58.3,critical,56.650000,용접기,60.0,안전
7,8,M002,2024-01-01 09:15,40.0,normal,41.050000,프레스B,45.0,안전


### status별 로그 개수 구하기

In [7]:
sensor_logs['status'].value_counts()

status
normal      4
warning     2
critical    2
Name: count, dtype: int64

### 기계별 평균 온도가 허용 최대 온도의 80%를 넘는 기계 찾기

In [8]:
# 기계 id별 평균 온도 구하기
# machine의 정보들이랑 merge하기 
# 80% 조건 걸어서 추출하기 

sensor_logs['machine_temp_mean'] = sensor_logs.groupby('machine_id',as_index=False)['temperature'].transform('mean') # 🐚 Groupby index조심.  as_index=False
sensor_logs.sort_values('machine_id',ascending=True)

merge_table = pd.merge(sensor_logs,machines,how='inner',on='machine_id')
merge_table

merge_table_80per = merge_table[(merge_table['machine_temp_mean'] / merge_table['max_temp']) > 0.8]
print(merge_table_80per)

   log_id machine_id         timestamp  temperature    status  \
0       1       M001  2024-01-01 09:00         35.2    normal   
1       2       M001  2024-01-01 09:05         38.5    normal   
2       3       M002  2024-01-01 09:00         42.1   warning   
3       4       M002  2024-01-01 09:10          NaN   warning   
4       5       M001  2024-01-01 09:10         36.0    normal   
5       6       M003  2024-01-01 09:00         55.0  critical   
6       7       M003  2024-01-01 09:05         58.3  critical   
7       8       M002  2024-01-01 09:15         40.0    normal   

   temperature_mean  machine_temp_mean machine_name  max_temp  
0         36.566667          36.566667         프레스A      40.0  
1         36.566667          36.566667         프레스A      40.0  
2         41.050000          41.050000         프레스B      45.0  
3         41.050000          41.050000         프레스B      45.0  
4         36.566667          36.566667         프레스A      40.0  
5         56.650000          5